In [ ]:
# HIDDEN
from datascience import *
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

## Review

In [ ]:
def standard_units(arr):
    """ Converts an array to standard units """
    return (arr - np.average(arr))/np.std(arr)

def correlation(t, x, y):
    """ Computes correlation: t is a table, and x and y are column names """
    x_standard = standard_units(t.column(x))
    y_standard = standard_units(t.column(y))
    return np.average(x_standard * y_standard)

def slope(t, x, y):
    """ Computes the slope of the regression line, like correlation above """
    r = correlation(t, x, y)
    y_sd = np.std(t.column(y))
    x_sd = np.std(t.column(x))
    return r * y_sd / x_sd

def intercept(t, x, y):
    """ Computes the intercept of the regression line, like slope above """
    x_mean = np.mean(t.column(x))
    y_mean = np.mean(t.column(y))
    return y_mean - slope(t, x, y)*x_mean

def fitted_values(t, x, y):
    """Return an array of the regression estimates (predictions) at all the x values"""
    a = slope(t, x, y)
    b = intercept(t, x, y)
    return a*t.column(x) + b

_________

In [ ]:
demographics = Table.read_table('district_demographics2016.csv')
demographics.sample(6).show()

**Linear Fit**

$\text{estimate of Median Income} = a \cdot \text{College\%} + b$

In [ ]:
fitted_income = demographics.select('College%', 'Median Income')
fitted_income = fitted_income.with_columns('Fitted',
    fitted_values(demographics, 'College%', 'Median Income'))
fitted_income.scatter('College%')

In [ ]:
def local_average_income(college):
    """Return a prediction of Median Income from College%
    by averaging over districts with similar College% values.
    """
    a_little = 2
    close_points = demographics.where('College%', are.between(college - a_little, 
                                                              college + a_little))
    return close_points.column('Median Income').mean()   

with_averages = fitted_income.with_columns('Graph of Averages',
    demographics.apply(local_average_income, 'College%'))

with_averages.scatter('College%')

## New material

In [ ]:
def residuals(t, x, y):
    predictions = fitted_values(t, x, y)
    return t.column(y) - predictions

### Residual plots can tell us about our model fit

#### Case study 1: 2016 election demographics

In [ ]:
fitted_income = demographics.select('College%', 'Median Income')
fitted_income = fitted_income.with_columns('Fitted',
    fitted_values(demographics, 'College%', 'Median Income'))
fitted_income.scatter('College%')

In [ ]:
income_residuals = fitted_income.with_columns(
    'Residual', residuals(demographics, 'College%', 'Median Income')
)
income_residuals.show(5)

These two functions will help us plot things the rest of the way.

In [ ]:
def plot_fitted(t, x, y):
    tbl = t.with_columns('Fitted', fitted_values(t, x, y))
    tbl.select(x, y, 'Fitted').scatter(0)

In [ ]:
def plot_residuals(t, x, y):
    tbl = t.with_columns('Residual', residuals(t, x, y))
    tbl.scatter(x, 'Residual')

______

In [ ]:
plot_fitted(demographics, 'College%', 'Median Income')

In [ ]:
plot_residuals(demographics, 'College%', 'Median Income')

#### Case study 2: dugongs 

In [ ]:
dugong = Table.read_table('dugong.csv')
dugong.show(5)

In [ ]:
dugong.scatter('Length', 'Age')

In [ ]:
correlation(dugong, 'Length', 'Age')

$\text{estimate of Age} = a \cdot \text{Length} + b$

In [ ]:
plot_fitted(dugong, 'Length', 'Age')

In [ ]:
plot_residuals(dugong, 'Length', 'Age')

#### Case study 3: US women

In [ ]:
us_women = Table.read_table('us_women.csv')
us_women.show(5)

In [ ]:
us_women.scatter('height')

In [ ]:
correlation(us_women, 'height', 'ave weight')

$\text{estimate of height} = a \cdot \text{average weight} + b$

In [ ]:
plot_fitted(us_women, 'height', 'ave weight')

In [ ]:
plot_residuals(us_women, 'height', 'ave weight')

#### Case study 4: Heights

In [ ]:
family_heights = Table.read_table('family_heights.csv')
family_heights.where('family', '1')

In [ ]:
parents = (family_heights.column('father') + family_heights.column('mother'))/2
heights = Table().with_columns(
    'Parent Average', parents,
    'Child', family_heights.column('child')
    )
heights.show(6)

$\text{estimate of a child's adult height} = a \cdot \text{average parent height} + b$

In [ ]:
plot_fitted(heights, 'Parent Average', 'Child')

In [ ]:
plot_residuals(heights, 'Parent Average', 'Child')

### Residuals have some special properties!

 The average of the residuals are zero.

In [ ]:
round(np.average(residuals(demographics, 'College%', 'Median Income')), 10)

In [ ]:
round(np.average(residuals(dugong, 'Length', 'Age')), 10)

In [ ]:
round(np.average(residuals(heights, 'Parent Average', 'Child')), 10)

______

The correlation coefficients between:

- The residuals and $x$
- The residuals and the fitted values

are both 0.

In [ ]:
heights = heights.with_columns(
    'Residual', residuals(heights, 'Parent Average', 'Child'),
    'Fitted Value', fitted_values(heights, 'Parent Average', 'Child')
)

In [ ]:
round(correlation(heights, 'Parent Average', 'Residual'), 10)

In [ ]:
round(correlation(heights, 'Fitted Value', 'Residual'), 10)